In [1]:
import yfinance as yf
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os
import json
import pandas_ta as ta
import talib as ta


In [7]:
# TALIB Calculations
def fetch_and_calculate_indicators(ticker):

    end_date = datetime.now().strftime('%Y-%m-%d')
    
    # Fetch historical data with dynamic end date
    data = yf.download(ticker, start='2023-01-01', end=end_date)
    
    # Drop rows with missing data
    data.dropna(inplace=True)

    # Calculate all indicators and store in local variables
    sma_20 = ta.SMA(data['Close'], timeperiod=20)[-1]
    ema_20 = ta.EMA(data['Close'], timeperiod=20)[-1]
    rsi_14 = ta.RSI(data['Close'], timeperiod=14)[-1]
    
    # MACD
    macd_line = ta.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)[0][-1]
    macd_signal = ta.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)[1][-1]
    
    # Bollinger Bands
    upper_bb = ta.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)[0][-1]
    lower_bb = ta.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)[2][-1]
    
    # Stochastic Oscillator
    slowk = ta.STOCH(data['High'], data['Low'], data['Close'], 
                     fastk_period=14, slowk_period=3, slowk_matype=0, 
                     slowd_period=3, slowd_matype=0)[0][-1]
    slowd = ta.STOCH(data['High'], data['Low'], data['Close'], 
                     fastk_period=14, slowk_period=3, slowk_matype=0, 
                     slowd_period=3, slowd_matype=0)[1][-1]
    
    obv = ta.OBV(data['Close'], data['Volume'])[-1]
    atr = ta.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)[-1]
    cci = ta.CCI(data['High'], data['Low'], data['Close'], timeperiod=20)[-1]
    sar = ta.SAR(data['High'], data['Low'], acceleration=0.02, maximum=0.2)[-1]
    roc = ta.ROC(data['Close'], timeperiod=12)[-1]
    adx = ta.ADX(data['High'], data['Low'], data['Close'], timeperiod=14)[-1]

    ma = 1 if ema_20 > sma_20 else 0

    # Individual signals with weights (adjust weights based on your strategy)
    signals = {
        # Trend Signals (40% total)
        'ma_trend': {'signal': 1 if ema_20 > sma_20 else 0, 'weight': 0.15},
        'macd_trend': {'signal': 1 if macd_line > macd_signal else 0, 'weight': 0.15},
        'adx_trend': {'signal': min(adx/50, 1) * (1 if ema_20 > sma_20 else -1), 'weight': 0.10},
        
        # Momentum Signals (30% total)
        'rsi_signal': {'signal': (rsi_14 - 50) / 50, 'weight': 0.10},  # Normalized to [-1, 1]
        'stoch_signal': {'signal': (((slowk + slowd) / 2) - 50) / 50, 'weight': 0.10},  # Normalized to [-1, 1]
        'roc_signal': {'signal': min(max(roc/10, -1), 1), 'weight': 0.10},  # Capped at [-1, 1]
        
        # Volatility Signals (20% total)
        'bb_signal': {'signal': 1 if data['Close'][-1] < lower_bb else (-1 if data['Close'][-1] > upper_bb else 0), 'weight': 0.10},
        'atr_signal': {'signal': min(atr / data['Close'][-1], 0.1) * 10, 'weight': 0.10},  # Normalized and scaled
        
        # Volume Signals (10% total)
        'obv_signal': {'signal': 1 if obv > 0 else -1, 'weight': 0.05},
        'cci_signal': {'signal': min(max(cci/100, -1), 1), 'weight': 0.05}  # Normalized to [-1, 1]
    }
    
    # Calculate weighted average
    weighted_signal = sum(item['signal'] * item['weight'] for item in signals.values())
    
    # Normalize to [-1, 1] range
    weighted_signal = min(max(weighted_signal, -1), 1)
    
    # Optional: Add confidence level based on signal strength
    confidence = abs(weighted_signal)
    
    return weighted_signal, confidence

In [5]:
def process_ticker(ticker, min_price=5, min_volume=100000):
    """Process single ticker and return result"""
    try:
        # Get stock info and current price
        stock = yf.Ticker(ticker)
        current_data = stock.history(period='1d')
        if current_data.empty:
            return None
            
        price = current_data['Close'].iloc[-1]
        volume = current_data['Volume'].iloc[-1]
        
        # Skip if doesn't meet criteria
        if price < min_price or volume < min_volume:
            return None
            
        try:
            signal, confidence = fetch_and_calculate_indicators(ticker)
        except Exception as e:
            # print(f"Error calculating indicators for {ticker}: {str(e)}")
            return None
        
        return {
            'ticker': ticker,
            'signal': signal,
            'confidence': confidence,
            'price': price,
            'volume': volume,
            'name': stock.info.get('shortName', ticker)
        }
        
    except Exception as e:
        # print(f"Error processing {ticker}: {str(e)}")
        return None

In [15]:
def get_top_trading_signals(min_price=5, min_volume=100000, top_n=100):
    """
    Get top 100 trading signals for US stocks
    Parameters:
        min_price: minimum stock price to consider
        min_volume: minimum average daily volume to consider
        top_n: number of top signals to return
    Returns:
        List of tuples (ticker, signal, confidence)
    """
    # Load tickers from JSON
    with open('company_tickers.json', 'r') as f:
        ticker_data = json.load(f)
    
    us_tickers = [data['ticker'] for data in ticker_data.values()]
    
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(process_ticker, ticker, min_price, min_volume) 
                  for ticker in us_tickers]
        
        for future in tqdm(futures, total=len(us_tickers), desc="Processing stocks"):
            try:
                result = future.result()
                if result and result['signal'] > 0:  # Only include buy signals
                    results.append((
                        result['ticker'],
                        result['signal'],
                        result['confidence']
                    ))
            except Exception as e:
                continue
    
    # Sort by signal strength * confidence
    ranked_results = sorted(results, 
                          key=lambda x: x[1] * x[2], 
                          reverse=True)
    
    # Return top N results
    return ranked_results[:top_n]

In [ ]:
def print_top_signals():
    top_signals = get_top_trading_signals(top_n=100)
    print("\nTop 100 Buy Signals:")
    print("Rank  Ticker  Signal  Confidence  Score")
    print("-" * 45)
    for i, (ticker, signal, conf) in enumerate(top_signals, 1):
        score = signal * conf
        print(f"{i:3d}.  {ticker:6s}  {signal:+.2f}    {conf:.1%}      {score:.3f}")

# Run the analysis
if __name__ == "__main__":
    print_top_signals()

In [8]:
print(fetch_and_calculate_indicators('ISRG'))

[*********************100%***********************]  1 of 1 completed

(0.557043299899093, 0.557043299899093)
